Goal: Make customer support agent

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QPBEqKSigNIzgPTpNJtM_XVysbHhyetc?usp=sharing)

In [1]:
!pip3 install -q crewai crewai_tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

# Setup

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os

from crewai import Agent, Task, Crew

In [3]:
# Add your utilities or helper functions to this file.

import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def get_serper_api_key():
    load_env()
    openai_api_key = os.getenv("SERPER_API_KEY")
    return openai_api_key


# break line every 80 characters if line is longer than 80 characters
# don't break in the middle of a word
def pretty_print_result(result):
  parsed_result = []
  for line in result.split('\n'):
      if len(line) > 80:
          words = line.split(' ')
          new_line = ''
          for word in words:
              if len(new_line) + len(word) + 1 > 80:
                  parsed_result.append(new_line)
                  new_line = word
              else:
                  if new_line == '':
                      new_line = word
                  else:
                      new_line += ' ' + word
          parsed_result.append(new_line)
      else:
          parsed_result.append(line)
  return "\n".join(parsed_result)


In [4]:
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

# Role Playing, Focus and Cooperation
- Detail for each agent
    - role : The agent is assigned to the specific role
    - goal : The goal sets the agent’s mission
    - Backstory : This is additional context for the agent, detailing its fictional background and approach.
    - Allow deligation : Specifies whether this agent can delegate tasks to other agents or systems.
    - Verbose : Setting `verbose=True` makes the agent more likely to provide detailed responses or log additional information.



- Concept Behind Multi Agent
    - Role Playing : Both agents have been given a role, goal and backstory
    - Focus : Both agents have been prompted to get into the characrer of the roles they are playing
    - Cooperation : Suppport Quality Assurance Agent can delegate work back to the Support Agent, Allowing for these agents to work together

In [5]:
# support agent
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

# Quality assurance agent
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

# Tools
- Possible Custom Tools
    - Load customer data
    - Tap into previous conversations
    - Load data from CRM
    - Checking existing bug reports
    - Checking existing feature requests
    - Checking ongoing tickets

- How to use crewai toools
    - `search_tool = SerperDevTool()`
    - `scrape_tool = ScrapeWebsiteTool()`

- Ways to give agents tools
    - Agent Level: The agent can use the tools on any task it performs
    - Task Level : The agent will only use the tools when  performing that specific task

In [6]:
# Tools
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

# web scrape tool
docs_scrape_tool = ScrapeWebsiteTool(
    website_url = "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

# Creating task

In [7]:
# passing tools on the task level
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [8]:
# QA Agent review the work of the support agent and not using any tools
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

# Create Crew

In [10]:
# Adding memory
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

In [12]:
# Guardrails : adding detail guideline
inputs = {
    "customer": "Kalbe Digital Lab",
    "person": "Adhi Setiawan",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: Kalbe Digital Lab just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Adhi Setiawan from Kalbe Digital Lab is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: Thought: I need to access the specific documentation from the CrewAI website to provide detailed guidance for Adhi Setiawan from Kalbe Digital Lab on setting up a Crew and adding memory to it.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Introduction - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationGet StartedIntroductionGet StartedExamplesCrewAI home pageCommunityChangelogGet StartedIntroductionInstallat

In [14]:
def extract_markdown_from_crew(content):
    """
    Extracts raw markdown text from CrewOutput object.

    Args:
        content: CrewOutput object or string

    Returns:
        str: The raw markdown text
    """
    try:
        # If it's a CrewOutput object, extract the raw content
        if hasattr(content, 'raw'):
            return content.raw
        # If it's already a string, return it directly
        return str(content)

    except Exception as e:
        raise Exception(f"Error extracting markdown: {str(e)}")


# Extract the markdown
try:
  markdown_text = extract_markdown_from_crew(result)
  print(markdown_text)  # This will print the raw markdown text
except Exception as e:
  print(f"Error: {e}")

Hello Adhi Setiawan,

Thank you for reaching out about setting up a Crew and adding memory functions in CrewAI. I'm delighted to assist you with this!

While I couldn't find specific step-by-step instructions directly from the CrewAI documentation right now, I'm here to provide some guidance based on general practices within CrewAI. Here’s how you can proceed:

1. **Create Your Crew:**
   - Start by defining your Crew within CrewAI. This means specifying the roles and responsibilities for each agent involved. Think about the tasks each agent will handle and ensure their roles are clear and well-defined.

2. **Integrate Memory:**
   - Memory functions often involve creating memory components for your agents in CrewAI. These memory components can include states or databases that agents can access and update during their operations. It's like giving them a "memory" to remember past actions or stored data.

3. **Utilize Memory Tools:**
   - If CrewAI includes built-in memory tools or APIs,